# 使用定位器结果的分类器
这里是localizer方法下的分类器。
输入是localizer截出来的鱼的子图（Nx3x100x100），理想情况就是刚刚好的一条鱼。输出是分类器分好的各个类的分数（Nx8）。

In [2]:
import os
import h5py
import numpy as np
np.random.seed(2017)
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
K.set_image_dim_ordering('th')


Using TensorFlow backend.


In [3]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D, AveragePooling2D
from keras.optimizers import SGD, Adagrad
from keras.utils import np_utils
from keras.constraints import maxnorm
from sklearn.metrics import log_loss
from keras import __version__ as keras_version

In [4]:
from keras.callbacks import EarlyStopping

In [5]:
weights_path = '../deep-learning-models/vgg16_weights.h5'
top_model_weights_path = 'classification/class_bottleneck_fc_model.h5'
# dimensions of our images.0
img_width, img_height = 100, 100
nb_epoch = 10

In [6]:
def load_from_file(filename):
	import numpy as np
	return np.load( filename + '.npy')

In [7]:
train_target = load_from_file('classification/train_target_100')
train_data = load_from_file('classification/train_data_100')

In [10]:
train_data.shape

(3764, 3, 100, 100)

In [11]:
def get_train_val():
    train_target = load_from_file('classification/train_target_100')
    train_data = load_from_file('classification/train_data_100')
    return train_data, train_target

# Step1
Fine Tune第一步是使用已经训练好的vgg16模型提取features.即将vgg16最后一个convBlock的输出作为features.

In [15]:
def save_bottleneck_features():
    # build the VGG16 network
    model = Sequential()
    model.add(ZeroPadding2D((1, 1), input_shape=(3, img_width, img_height)))

    model.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_1', dim_ordering='th'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_2', dim_ordering='th'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_1', dim_ordering='th'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_2', dim_ordering='th'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_1', dim_ordering='th'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_2', dim_ordering='th'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_3', dim_ordering='th'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_1', dim_ordering='th'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_2', dim_ordering='th'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_3', dim_ordering='th'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_1', dim_ordering='th'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_2', dim_ordering='th'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_3', dim_ordering='th'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # load the weights of the VGG16 networks
    # (trained on ImageNet, won the ILSVRC competition in 2014)
    # note: when there is a complete match between your model definition
    # and your weight savefile, you can simply call model.load_weights(filename)
    assert os.path.exists(weights_path), 'Model weights not found (see "weights_path" variable in script).'
    f = h5py.File(weights_path)
    for k in range(f.attrs['nb_layers']):
        if k >= len(model.layers):
            # we don't look at the last (fully-connected) layers in the savefile
            break
        g = f['layer_{}'.format(k)]
        weights = [g['param_{}'.format(p)] for p in range(g.attrs['nb_params'])]
        model.layers[k].set_weights(weights)
    f.close()
    print('Model loaded.')
    # data generator:
    X, Y = get_train_val()
    # train:
    bottleneck_features_train = model.predict(X[330:], batch_size=32, verbose=0)
    np.save(open('classification/local_bottleneck_features_train.npy', 'w'), bottleneck_features_train)
    # validation:
    bottleneck_features_val = model.predict(X[:330], batch_size=32, verbose=0)
    np.save(open('classification/local_bottleneck_features_val.npy', 'w'), bottleneck_features_val)
    

In [16]:
save_bottleneck_features()

Model loaded.


# Step 2
Fine Tune第二步是使用第一步提取的特征训练一个自定义的适用于自己要解决的问题的head.这里是训练分类器，所以是classification head。
将训练好的weights保存作为正式进行训练时的初始值。

In [18]:
def train_top_model():
    from keras.callbacks import EarlyStopping
    train_target = load_from_file('classification/train_target_100')
    
    train_data = np.load(open('classification/local_bottleneck_features_train.npy'))
    train_labels = train_target[330:]

    validation_data = np.load(open('classification/local_bottleneck_features_val.npy'))
    validation_labels = train_target[:330]

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(96, activation='relu',init='he_uniform'))
    model.add(Dropout(0.4))
    model.add(Dense(24, activation='relu',init='he_uniform'))
    model.add(Dropout(0.2))
    model.add(Dense(8, activation='softmax'))

    sgd = SGD(lr=1e-2, decay=1e-4, momentum=0.89, nesterov=False)
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
    callbacks = [
            EarlyStopping(monitor='val_loss', patience=3, verbose=0),
        ]
    model.fit(train_data, train_labels,
              nb_epoch=nb_epoch, batch_size=32, shuffle=True, verbose=2, 
              validation_data=(validation_data, validation_labels), callbacks = callbacks)
    model.save_weights(top_model_weights_path)

In [21]:
train_top_model()

Train on 3434 samples, validate on 330 samples
Epoch 1/10
2s - loss: 1.7856 - acc: 0.3888 - val_loss: 0.5598 - val_acc: 1.0000
Epoch 2/10
1s - loss: 1.5010 - acc: 0.4694 - val_loss: 0.9134 - val_acc: 0.9879
Epoch 3/10
1s - loss: 1.4476 - acc: 0.4802 - val_loss: 0.8834 - val_acc: 1.0000
Epoch 4/10
1s - loss: 1.4193 - acc: 0.4988 - val_loss: 0.6802 - val_acc: 1.0000
Epoch 5/10
1s - loss: 1.4016 - acc: 0.4965 - val_loss: 0.6192 - val_acc: 0.9939


# Step 3
Fine Tune第三步是利用第二步训练好的头部weights和vgg16模型本身的weights在自己问题上的数据集进行训练。
这里数据规模中等的情况下freeze掉前四层convblock，对最后一个convblock和头部进行权值更新。

In [12]:
# build the VGG16 network
model = Sequential()
model.add(ZeroPadding2D((1, 1), input_shape=(3, img_width, img_height)))

model.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_2'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_2'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_2'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

In [23]:
assert os.path.exists(weights_path), 'Model weights not found (see "weights_path" variable in script).'
f = h5py.File(weights_path)
for k in range(f.attrs['nb_layers']):
    if k >= len(model.layers):
        # we don't look at the last (fully-connected) layers in the savefile
        break
    g = f['layer_{}'.format(k)]
    weights = [g['param_{}'.format(p)] for p in range(g.attrs['nb_params'])]
    model.layers[k].set_weights(weights)
f.close()
print('Model loaded.')

Model loaded.


In [13]:
# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=model.output_shape[1:]))
top_model.add(Dense(96, activation='relu',init='he_uniform'))
top_model.add(Dropout(0.4))
top_model.add(Dense(24, activation='relu',init='he_uniform'))
top_model.add(Dropout(0.2))
top_model.add(Dense(8, activation='softmax'))
# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning
top_model.load_weights(top_model_weights_path)

In [14]:
# add the model on top of the convolutional base
model.add(top_model)

In [16]:
# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:25]:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
sgd = SGD(lr=1e-4, decay=1e-6, momentum=0.9, nesterov=False)

model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

In [27]:
# fine tune:
X, Y = get_train_val()
train_data = X[330:]
train_labels = Y[330:]
validation_data = X[:330]
validation_labels = Y[:330]
callbacks = [
            EarlyStopping(monitor='val_loss', patience=3, verbose=0)
        ]
model.fit(train_data, train_labels,
              nb_epoch=nb_epoch, batch_size=32, shuffle=True, verbose=2,
              validation_data=(validation_data, validation_labels), callbacks=callbacks)
    
model.save_weights('classification/local-fine-tune-model.h5')

Train on 3434 samples, validate on 330 samples
Epoch 1/10
1592s - loss: 1.3282 - acc: 0.5111 - val_loss: 0.5808 - val_acc: 1.0000
Epoch 2/10
1577s - loss: 1.3252 - acc: 0.5116 - val_loss: 0.5738 - val_acc: 1.0000
Epoch 3/10
2952s - loss: 1.3006 - acc: 0.5122 - val_loss: 0.5601 - val_acc: 1.0000
Epoch 4/10
2827s - loss: 1.3066 - acc: 0.5099 - val_loss: 0.5328 - val_acc: 1.0000
Epoch 5/10
2872s - loss: 1.2890 - acc: 0.5140 - val_loss: 0.5052 - val_acc: 1.0000
Epoch 6/10
2817s - loss: 1.2758 - acc: 0.5149 - val_loss: 0.5051 - val_acc: 1.0000
Epoch 7/10
2747s - loss: 1.2760 - acc: 0.5154 - val_loss: 0.5022 - val_acc: 1.0000
Epoch 8/10
2940s - loss: 1.2667 - acc: 0.5151 - val_loss: 0.4949 - val_acc: 1.0000
Epoch 9/10
3119s - loss: 1.2708 - acc: 0.5154 - val_loss: 0.4313 - val_acc: 1.0000
Epoch 10/10
2930s - loss: 1.2527 - acc: 0.5178 - val_loss: 0.4261 - val_acc: 1.0000


In [30]:
model.load_weights('classification/local-fine-tune-model.h5')
print 'model loaded'
sgd = SGD(lr=1e-3, decay=1e-5, momentum=0.9, nesterov=False)

model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
callbacks = [
            EarlyStopping(monitor='val_loss', patience=3, verbose=0)
        ]

model.fit(train_data, train_labels,
              nb_epoch=nb_epoch, batch_size=32, shuffle=True, verbose=2,
              validation_data=(validation_data, validation_labels), callbacks=callbacks)
    
model.save_weights('classification/local-fine-tune-model_second10epoch.h5')

model loaded
Train on 3434 samples, validate on 330 samples
Epoch 1/10
3001s - loss: 1.3073 - acc: 0.5111 - val_loss: 0.3298 - val_acc: 1.0000
Epoch 2/10
2833s - loss: 1.2361 - acc: 0.5527 - val_loss: 0.5236 - val_acc: 0.9394
Epoch 3/10
2777s - loss: 1.1992 - acc: 0.5789 - val_loss: 0.2080 - val_acc: 0.9788
Epoch 4/10
2809s - loss: 1.1292 - acc: 0.6095 - val_loss: 0.3828 - val_acc: 0.9606
Epoch 5/10
2743s - loss: 1.0011 - acc: 0.6471 - val_loss: 0.3565 - val_acc: 0.9030
Epoch 6/10
2752s - loss: 0.9260 - acc: 0.6744 - val_loss: 0.0891 - val_acc: 0.9848
Epoch 7/10
2779s - loss: 0.8419 - acc: 0.7053 - val_loss: 0.1874 - val_acc: 0.9606
Epoch 8/10
2755s - loss: 0.7352 - acc: 0.7242 - val_loss: 0.1595 - val_acc: 0.9576
Epoch 9/10
2753s - loss: 0.7493 - acc: 0.7289 - val_loss: 0.3716 - val_acc: 0.9182
Epoch 10/10
2749s - loss: 0.6129 - acc: 0.7662 - val_loss: 0.2646 - val_acc: 0.9364


In [31]:
model.fit(train_data, train_labels,
              nb_epoch=nb_epoch, batch_size=32, shuffle=True, verbose=2,
              validation_data=(validation_data, validation_labels), callbacks=callbacks)
    
model.save_weights('classification/local-fine-tune-model_third10epoch.h5')

Train on 3434 samples, validate on 330 samples
Epoch 1/10
2895s - loss: 0.5864 - acc: 0.7854 - val_loss: 0.2466 - val_acc: 0.9424
Epoch 2/10
2975s - loss: 0.4885 - acc: 0.8241 - val_loss: 0.2202 - val_acc: 0.9576
Epoch 3/10
3117s - loss: 0.4089 - acc: 0.8559 - val_loss: 0.1040 - val_acc: 0.9909
Epoch 4/10
2836s - loss: 0.3603 - acc: 0.8727 - val_loss: 0.6311 - val_acc: 0.7818
Epoch 5/10
2866s - loss: 0.3236 - acc: 0.8925 - val_loss: 0.3231 - val_acc: 0.8939
Epoch 6/10
2897s - loss: 0.2735 - acc: 0.9077 - val_loss: 0.1101 - val_acc: 0.9758
Epoch 7/10
2756s - loss: 0.2693 - acc: 0.9086 - val_loss: 0.3054 - val_acc: 0.9273


In [17]:
# fine tune:
X, Y = get_train_val()
train_data = X[330:]
train_labels = Y[330:]
validation_data = X[:330]
validation_labels = Y[:330]

In [18]:
train_data

array([[[[ 0.33333334,  0.31764707,  0.3137255 , ...,  0.18039216,
           0.19215687,  0.1882353 ],
         [ 0.32941177,  0.32941177,  0.32941177, ...,  0.1882353 ,
           0.18039216,  0.18039216],
         [ 0.32156864,  0.32549021,  0.33333334, ...,  0.27843139,
           0.2       ,  0.17647059],
         ..., 
         [ 0.37254903,  0.36078432,  0.35686275, ...,  0.67058825,
           0.25882354,  0.18431373],
         [ 0.3764706 ,  0.36862746,  0.36470589, ...,  0.69411767,
           0.27843139,  0.19215687],
         [ 0.38431373,  0.3764706 ,  0.37254903, ...,  0.65882355,
           0.34901962,  0.15686275]],

        [[ 0.33725491,  0.33725491,  0.32549021, ...,  0.17254902,
           0.18431373,  0.18039216],
         [ 0.34117648,  0.34509805,  0.34509805, ...,  0.18039216,
           0.17254902,  0.17254902],
         [ 0.34117648,  0.34901962,  0.35294119, ...,  0.27058825,
           0.19215687,  0.16862746],
         ..., 
         [ 0.37254903,  0.360784

# Data Augmentation
引入Data Augmentation可以防止对原始数据的过拟合，更generalize model。

In [20]:
# data augmentation:
sgd = SGD(lr=1e-3, decay=1e-5, momentum=0.9, nesterov=False)

model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

callbacks = [
            EarlyStopping(monitor='val_loss', patience=3, verbose=0)
        ]

# prepare data augmentation configuration
datagen = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
datagen.fit(train_data)

nb_epoch = 10
model.fit_generator(datagen.flow(train_data, train_labels, batch_size=32),
                            samples_per_epoch=train_data.shape[0],
                            nb_epoch=nb_epoch,
                            validation_data=(validation_data, validation_labels),
                            callbacks=callbacks, 
                    )


model.save_weights('classification/local-fine-tune-model_da_forth10epoch.h5')


Epoch 1/10
3434/3434 [==============================] - 1525s - loss: 1.1879 - acc: 0.6252 - val_loss: 0.0937 - val_acc: 0.9939
Epoch 2/10
3434/3434 [==============================] - 1524s - loss: 0.9270 - acc: 0.6928 - val_loss: 0.1777 - val_acc: 0.9455
Epoch 3/10
3434/3434 [==============================] - 4647s - loss: 0.8288 - acc: 0.7368 - val_loss: 0.2506 - val_acc: 0.9455
Epoch 4/10
3434/3434 [==============================] - 2833s - loss: 0.7535 - acc: 0.7519 - val_loss: 0.1974 - val_acc: 0.9394
Epoch 5/10
3434/3434 [==============================] - 2792s - loss: 0.7133 - acc: 0.7627 - val_loss: 0.1120 - val_acc: 0.9667
